# Part 07: Identify streets that cross sectors, plot individual streets
michael babb  
2025 02 18 

In [ ]:
# standard
import os
import re
import collections

In [ ]:
# external
import geopandas as gpd
import matplotlib as mpl
from matplotlib.gridspec import GridSpec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
# custom
import run_constants as rc
from utils import *

In [ ]:
# process flow variables

# load the street network data

In [ ]:
rc.S03_SND_WORKING_IN_FILE_NAME

In [ ]:
# load the list of streets - this is before any work is done to compute the missing segments. 
fpn = os.path.join(rc.OUTPUT_FILE_PATH, rc.S03_SND_WORKING_IN_FILE_NAME)

In [ ]:
gdf = gpd.read_file(filename = fpn)

In [ ]:
gdf.columns

In [ ]:
gdf['n_roads'] = int(1)

In [ ]:

gdf['ord_stname_concat'].unique().shape

In [ ]:
gdf['ord_stname_unique'].unique().shape

# group data by street type for use in version 2.0

In [ ]:
col_names = ['ord_street_name', 'ord_street_type', 'ord_stname_concat','ord_stname_unique',  'city_sector']
st_name_type_df = gdf[col_names].drop_duplicates()

In [ ]:
my_counter = collections.Counter(st_name_type_df['ord_stname_unique'])


In [ ]:
my_counter.most_common()

In [ ]:
# the number of unique streets
st_name_type_df.shape

In [ ]:
st_name_type_df = st_name_type_df.sort_values(by = ['ord_street_name', 'ord_street_type', 'ord_stname_concat'])

In [ ]:
st_name_type_df.head(n=10)

In [ ]:
# make some ids - ord_street_name
osn_df = st_name_type_df['ord_street_name'].drop_duplicates().to_frame().sort_values(by = 'ord_street_name').reset_index(drop = True)
osn_df['osn_id'] = range(0, osn_df.shape[0])
osn_df.head()

In [ ]:
# ord_street_name and ord_street_type
col_names = ['ord_street_name', 'ord_street_type']
osn_ost_df = st_name_type_df[col_names].drop_duplicates().sort_values(by = col_names).reset_index(drop = True)
osn_ost_df['osn_ost'] = osn_ost_df['ord_street_name'] + ' ' + osn_ost_df['ord_street_type']
osn_ost_df['osn_ost_id'] = range(0, osn_ost_df.shape[0])
osn_ost_df.head()


In [ ]:
# ord_stname_concat
ostc_df = st_name_type_df['ord_stname_concat'].drop_duplicates().to_frame().sort_values(by = 'ord_stname_concat').reset_index(drop = True)
ostc_df['ostc_id'] = range(0, ostc_df.shape[0])
ostc_df.head()


In [ ]:
# ord_stname_unique 
ostu_df = st_name_type_df['ord_stname_unique'].drop_duplicates().to_frame().sort_values(by = 'ord_stname_unique').reset_index(drop = True)
ostu_df['ostu_id'] = range(0, ostu_df.shape[0])
ostu_df.head()

In [ ]:
# join to get the id values
st_name_type_df = pd.merge(left = st_name_type_df, right = osn_df)
st_name_type_df = pd.merge(left = st_name_type_df, right = osn_ost_df)
st_name_type_df = pd.merge(left = st_name_type_df, right = ostc_df)
st_name_type_df = pd.merge(left = st_name_type_df, right = ostu_df)

In [ ]:
st_name_type_df.shape

In [ ]:
st_name_type_df.head()

In [ ]:
st_name_type_df['ord_street_type'].unique()

In [ ]:
st_name_type_df['city_sector'].unique()

In [ ]:
check = st_name_type_df.loc[st_name_type_df['city_sector'] == 'WB', ]

In [ ]:
check.head()

In [ ]:
# let's connect the ord_street_name and the ord_street_type
st_name_type_df['osn_ost'].unique().shape

In [ ]:
# LET'S COUNT HOW MANY STREETS WE NEED TO PLOT AND MANUALLY REVIEW

In [ ]:
st_name_type_df['n_roads'] = int(1)

In [ ]:
st_name_type_df.columns

In [ ]:
pv_df = pd.pivot_table(data = st_name_type_df, values = ['n_roads'],
                          index = ['ord_street_name', 'ord_street_type'],
                          columns = ['city_sector'], aggfunc='sum', fill_value=0,
                          margins=True)

In [ ]:
pv_df.head()

In [ ]:
# clean up column names
pv_df.columns = [cn[1] for cn in pv_df.columns]

In [ ]:
pv_df.columns

In [ ]:
pv_df = pv_df.reset_index()

In [ ]:
pv_df.tail()

In [ ]:
pv_df.shape

In [ ]:
# grouping by ord_street_name and ord_street_type tells us the number of streets
# there are across the city sectors
# we only need to worry about streets that span at least two sectors
osn_count_df = pv_df.loc[(pv_df['All'] >= 2), :].copy()

In [ ]:
# drop the All row
osn_count_df = osn_count_df.loc[osn_count_df['ord_street_name'] != 'All', :]

In [ ]:
osn_count_df.head()

In [ ]:
# melt, to reshape
osnt_df = pd.melt(frame = osn_count_df, id_vars = ['ord_street_name', 'ord_street_type'],var_name='city_sector', value_name = 'n_roads')

In [ ]:
# drop zero records
osnt_df = osnt_df.loc[(osnt_df['n_roads'] > 0), :].copy()

In [ ]:
osnt_df.head()

In [ ]:
# the osnt_df is effectively the same shape as the input gdf, but by pivoting we
# can very easily count the number of roads that span multiple sectors by using
# built-in tools. 

In [ ]:
osnt_df = osnt_df.loc[(osnt_df['city_sector'] != 'All'), :].copy()

In [ ]:
# sort it...
osnt_df = osnt_df.sort_values(by = ['ord_street_name', 'ord_street_type', 'city_sector'])

In [ ]:
# we don't need this column
osnt_df = osnt_df.drop(labels = ['n_roads'], axis = 1)

In [ ]:
osnt_df.head()

In [ ]:
# get the numeric value of the streets, for the sort order...
osnt_df['sort_order'] = osnt_df['ord_street_name'].map(get_sort_order)

In [ ]:
osnt_df.head()

In [ ]:
# reorder
col_names = ['sort_order', 'ord_street_name', 'ord_street_type', 'city_sector']
osnt_df = osnt_df[col_names].sort_values(by = col_names[:4])

In [ ]:
# set an index - this is helpful for when we write this to excel 
# easier to read for manual review
osnt_df = osnt_df.set_index(keys = col_names[:4])

In [ ]:
osnt_df.head()

In [ ]:
osnt_df.shape

In [ ]:
osnt_df.columns

In [ ]:
# save it to excel
ofpn = os.path.join(rc.ANALYSIS_OUTPUT_FILE_PATH, rc.S07_STREET_GROUP_OUT_FILE_NAME)
osnt_df.to_excel(excel_writer=ofpn)


# make a simple plot of each street. 

In [ ]:
# each plot will feature the city sector boundaries as created by the convex-hull 
# in step 2 and the different road segments for each street.

In [ ]:
# use the same colors for each city sector
cdm = {}
dir_list = ['CNTR', 'E', 'N', 'NE', 'NW', 'S', 'SW', 'W']
for idir, dir_value in enumerate(dir_list):
    cdm[dir_value] = mpl.colormaps["Dark2"].colors[idir]

my_cmap = mpl.colors.ListedColormap([cdm[c] for c in dir_list])

In [ ]:
# plot bounds to use across each plot
bounds = [-122.4197794277490061,47.4803548409661005, -122.2200188105690017,47.7341482423694004]

In [ ]:
# load the boundaries of the city sectors
ifpn = os.path.join(rc.OUTPUT_FILE_PATH, rc.S02_CITY_SECTORS_OUT_FILE_NAME)
cs_gdf = gpd.read_file(filename=ifpn)

In [ ]:
osnt_df = osnt_df.reset_index()

In [ ]:
osnt_df.head()

In [ ]:
osn_ost_df.shape

In [ ]:
wosnt_df = osnt_df[['ord_street_name', 'ord_street_type']].drop_duplicates()

In [ ]:
wosnt_df.shape

In [ ]:
t_wosnt_df = wosnt_df.iloc[1:10]

In [ ]:
t_wosnt_df.head()

In [ ]:
wosnt_df.shape

In [ ]:
make_plots = False
output_file_name_list = []
if make_plots:
    for irow, row in wosnt_df.iterrows():
        osn = row['ord_street_name']
        ost = row['ord_street_type']


        output_file_name = f"{osn}_{ost}.png"
        ofpn = os.path.join(rc.S06_PLOT_OUTPUT_FILE_PATH, output_file_name)
        print(ofpn)
        output_file_name_list.append(output_file_name)

        # now, let's make a map...
        wgdf = gdf.loc[(gdf['ord_street_name'] == osn) &
                    (gdf['ord_street_type'] == ost), :]
        to_draw = wgdf[['city_sector', 'geometry']].dissolve(by = 'city_sector', as_index = False)
        to_draw['coords'] = to_draw['geometry'].map(lambda x: x.centroid.coords[0])
        fig = plt.figure(layout = 'constrained', figsize = (5, 10))
        gs = GridSpec(1,1, figure = fig, height_ratios = [1])
        ax1 = fig.add_subplot(gs[0,0])
        ax1.set_xlim(bounds[0], bounds[2])
        ax1.set_ylim(bounds[1], bounds[3])
        cs_gdf.plot(ax = ax1, column = 'city_sector', alpha = .2)
        to_draw.plot(ax = ax1, column = 'city_sector', cmap = my_cmap,  linewidth = 5, legend = True)

        for irrow, row in to_draw.iterrows():    
            ax1.annotate(text=row['city_sector'], xy=row['coords'], fontsize = 16 )

        ax1.set_axis_off()

        osn = wgdf['ord_street_name'].unique().tolist()[0]
        ost = wgdf['ord_street_type'].unique().tolist()[0]
        my_title = f"Street Name: {osn} {ost}"
        plt.title(label = my_title)

        #plt.show()

        fig.savefig(fname = ofpn)
        plt.close()

# manually review each plot to identify the groups within each street.

In general, avenues run north-south and streets run east-west. 
Avenues can be grouped by:
* NE, E, CNTR, S  
* N, CNTR, S
* NW, W, SW

Streets can be be grouped by:
* NW, N, NE
* W, N, CNTR, E
* SW, S

# BONUS: is every state name represented as a street?

In [ ]:
fpn = os.path.join(rc.INPUT_FILE_PATH, 'state_name.csv')

In [ ]:
state_df = pd.read_csv(filepath_or_buffer=fpn)

In [ ]:
state_df.head()

In [ ]:
# build a set of the state names after cleaning and converting to lower case
state_name_set =  set([sn.strip().lower() for sn in state_df['Name']])

In [ ]:
len(state_name_set)

In [ ]:
# do the same for the street name after dropping the streets that start with a number.
street_name_set = set(gdf.loc[-gdf['ord_street_name'].str[0].isin(list('0123456789')), 'ord_street_name'].str.lower().unique().tolist())

In [ ]:
len(street_name_set)

In [ ]:
test_intersection = state_name_set.intersection(street_name_set)

In [ ]:
len(test_intersection)

In [ ]:
# so, 20 states are representated... 

In [ ]:
test_intersection

In [ ]:
# what's not represented?
state_name_set.difference(street_name_set)

In [ ]:
# what about states with directons in the name and DC?

In [ ]:
'dakota' in street_name_set

In [ ]:
'columbia' in street_name_set

In [ ]:
'virginia' in street_name_set

In [ ]:
'carolina' in street_name_set

In [ ]:
# if we count Dakota as ND and Sd
# columbia as Washington DC
# Virginia as both VA and West Virginia
# Carolina as both NC and SC
# that brings us to: 26. 
# so, over half. That's kind of cool.